In [24]:
%load_ext autoreload
%autoreload 2

import math

import matplotlib.pyplot as plt

from penai.hierarchy_generation.inference import HierarchyInferencer
from penai.hierarchy_generation.vis import (
    InteractiveHTMLHierarchyVisualizer,
    InteractiveSVGHierarchyVisualizer,
)
from penai.llm.llm_model import RegisteredLLM
from penai.registries.projects import SavedPenpotProject
from penai.registries.web_drivers import RegisteredWebDriver
from penai.utils.io import ResultWriter
from penai.utils.vis import ShapeVisualizer

plt.rcParams["figure.figsize"] = (20, 40)

RuntimeError: error checking inheritance of typing.Self (type: Self)

In [25]:
penpot_project = SavedPenpotProject.INTERACTIVE_MUSIC_APP.load()
main_file = penpot_project.get_main_file()
selected_page = main_file.get_page_by_name("Interactive music app")
selected_page.svg.remove_elements_with_no_visible_content()
selected_page.svg.retrieve_and_set_view_boxes_for_shape_elements(
    RegisteredWebDriver.CHROME,
)
playlist_shape = selected_page.svg.get_shape_by_name("Playlist")

Setting view boxes: 100%|██████████| 351/351 [00:03<00:00, 100.53it/s]


In [28]:
visualizer = ShapeVisualizer(RegisteredWebDriver.CHROME)
visualizations = list(visualizer.visualize_bboxes_in_shape(playlist_shape))

In [30]:
# plot_image_grid([vis.image for vis in visualizations])

In [31]:
hierarchy_inference = HierarchyInferencer(model=RegisteredLLM.GPT4O)
hierarchy, visualizations = hierarchy_inference.infer_shape_hierarchy(
    playlist_shape,
    return_visualizations=True,
)

44it [00:02, 18.40it/s]


```json
{
  "id": "1",
  "description": "Main container",
  "children": [
    {
      "id": "2",
      "description": "Header container",
      "children": [
        {
          "id": "3",
          "description": "Background image",
          "children": []
        },
        {
          "id": "4",
          "description": "Overlay rectangle",
          "children": [
            {
              "id": "5",
              "description": "Title text",
              "children": []
            },
            {
              "id": "6",
              "description": "Subtitle text",
              "children": []
            }
          ]
        },
        {
          "id": "44",
          "description": "Menu icon",
          "children": []
        }
      ]
    },
    {
      "id": "37",
      "description": "Control buttons container",
      "children": [
        {
          "id": "38",
          "description": "Play button",
          "children": [
            {
              "id": "39",
  

In [ ]:
visualizations = list(ShapeVisualizer().visualize_bboxes_in_shape(playlist_shape))

img_per_row = 4

fig, axes = plt.subplots(math.ceil(len(visualizations) / img_per_row), img_per_row)

for i, ax in enumerate(axes.flat):
    if i < len(visualizations):
        ax.imshow(visualizations[i].image)
    ax.axis("off")

plt.show()

In [ ]:
result_writer = ResultWriter(".")

hierarchy_svg_visualizer = InteractiveSVGHierarchyVisualizer(hierarchy, playlist_shape)
svg_path = result_writer.path("hierarchy.svg")
hierarchy_svg_visualizer.write_svg(svg_path)

hierarchy_html_visualizer = InteractiveHTMLHierarchyVisualizer(svg_path, hierarchy)
hierarchy_html_visualizer.write_html(result_writer.path("hierarchy.html"))